# Data Preparation for Neo4j

This notebook demonstrates how to prepare data for a Neo4j graph database from CSV files. It covers the necessary steps and techniques to transform and load the data into Neo4j, enabling efficient querying and analysis of the graph structure.

In [1]:
import pandas as pd 
data = pd.read_csv('data.csv')
data.head(3)

,ClaimID,Risk,Customer,Age,Gender,PhoneNumber,Email,Hospital,AdmissionDate,Disease,Los,AvgLos,Agent,Fraud,Narration
0,C0001,Low,Devon Q. White,56.0,Female,(345) 551-5593,839cb61c42@GMAIL.COM,Metropolis West Hospital #31,1-Apr-23,"Cutaneous abscess, furuncle and carbuncle of b...",8,7,Robin Y. King,No,"On April 1, 2023, a 56-year-old patient was ad..."
1,C0002,Low,Robin B. Lee,35.0,Female,(462) 538-4795,9656eb1f3a@GMAIL.COM,Astoria General Hospital #86,28-Dec-21,Peritonsilllar abscess,9,10,Robin Y. King,No,"On 28th December 2021, a 35-year-old patient w..."
2,C0003,Low,Skyler X. Jones,39.0,Female,(305) 975-5370,d55272309d@GMAIL.COM,Sunnydale East Hospital #95,4-Aug-21,Vertigo of central origin,7,7,Dale P. Allen,No,"On 4th August 2021, a 39-year-old patient was ..."


## Create embedding for `Narration`
Please refer to the [documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for more details.

In [ ]:
import os
from openai import OpenAI

# Assume that you have an api key set as an environment variable 'OPENAI_API_KEY'
llm = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

def generate_embeddings(text):
    response = llm.embeddings.create(
        input=text,
        model='text-embedding-3-small'
    )

    return  response.data[0].embedding

data['NarrationEmbedding'] = data['Narration'].apply(generate_embeddings)

In this notebook, we demonstrate how to import a CSV file into a Neo4j Desktop database. By following the steps outlined below, you will be able to efficiently load and query your data in a graph structure:

1. Identify the entities and relationships in your domain.
1. Define the node labels and relationship types that represent these entities and relationships.
1. Create a CSV file that contains the data for each node label and relationship type.
1. Ensure that each node has a unique identifier, such as an ID or a combination of properties that can uniquely identify it.
1. Map the properties of each node and relationship to the corresponding columns in the CSV file.
1. Use the Neo4j import tool or the Neo4j driver to load the data into your Neo4j Desktop database.
1. Verify the data has been successfully imported by querying the database.

By following these steps, you will be able to import your CSV data into a Neo4j Desktop database and leverage the power of graph querying and analysis.

For more information, please enroll  [Importing CSV data into Neo4j](https://graphacademy.neo4j.com/courses/importing-cypher/) 

## Prepare CSV Files

In [ ]:
# 1. Customer, Phone, Email and Relationship
data[['Customer','Gender']].drop_duplicates().to_csv('customers.csv',index=False)
data[['PhoneNumber']].drop_duplicates().to_csv('phones.csv',index=False)
data[['Email']].drop_duplicates().to_csv('emails.csv',index=False)

data[['Customer','PhoneNumber']].drop_duplicates().to_csv('owns_phone.csv',index=False)
data[['Customer','Email']].drop_duplicates().to_csv('owns_email.csv',index=False)

In [ ]:
# 2. Claim, Relationshipw with Customer, Claim embedding
data[['ClaimID','Risk','AdmissionDate','Age','Disease','Los','Fraud','Narration']].drop_duplicates().to_csv('claims.csv',index=False)
data[['Customer','ClaimID']].drop_duplicates().to_csv('filed_claim.csv',index=False)
data[['ClaimID','NarrationEmbedding']].drop_duplicates(subset=['ClaimID']).to_csv('openai_embedding_3small.csv',index=False)

In [ ]:
# 3. Hospital, Relationship with Claim
data[['Hospital']].drop_duplicates().to_csv('hospitals.csv',index=False)
data[['Hospital','ClaimID']].drop_duplicates().to_csv('provided_medical.csv',index=False)

In [ ]:
# 4. Agent, Relationship with Claim
data[['Agent']].drop_duplicates().to_csv('agents.csv',index=False)
data[['Agent','ClaimID']].drop_duplicates().to_csv('serviced_claim.csv',index=False)

# Setup Neo4j Graph Database

Follow instruction on [neo4j](https://neo4j.com/download/) to download and install Neo4j Desktop

Once, you have successfull install Neo4j on your system,
1. Create a new project

    <img src="../img/new-project.png" >

2. Under the new project, add `Local DMBS` 

    <img src="../img/add-local-dbms.png" >

3. Create a password for the database. To access the vector index feature, make sure you are using `version 5.13` or later. 

    <img src="../img/create-database.png" style="width:60%;height:60%;">

4. Once the database is created, install Plugin `APOC` and `Graph Data Science Library`

    <img src="../img/install-plugin.png" style="width:60%;height:60%;">

5. Click on the `...` of the database and open the `import` folder, then copy all csv files in to this folder. You are now able to access csv files within Neo4j file system

    <img src="../img/import.png" style="width:60%;height:60%;">

6. Start Neo4j Graph Database by clickinh `Start`. 

  <img src="../img/start.png" style="width:60%;height:60%;">

Once the database is `Active`, click `Open` to start `Neo4j Browser`


  <img src="../img/open.png" style="width:60%;height:60%;">


**Neo4j Browser**

<img src="../img/browser.png" style="width:60%;height:60%;">


## Populate Nodes and Relationships with Cypher script


<img src="../img/import-customers.png" style="width:60%;height:60%;">


You may use below Cypher script to import csv files data into the database

### 1. Customer, Phone, Email and Relationship 

```Cypher

// CREATE Customer
CREATE CONSTRAINT Customer_name IF NOT EXISTS 
FOR (x:Customer) REQUIRE x.name IS UNIQUE;
LOAD CSV WITH HEADERS FROM 'file:///customers.csv' AS row
MERGE (c:Customer {name: row.Customer})
SET 
	c.name = row.Customer,
	c.gender = row.Gender;
	
	
// CREATE Phone
CREATE CONSTRAINT Phone_phoneNumber IF NOT EXISTS 
FOR (x:Phone) REQUIRE x.phoneNumber IS UNIQUE;
LOAD CSV WITH HEADERS FROM 'file:///phones.csv' AS row
MERGE (p:Phone {phoneNumber: row.PhoneNumber})
SET 
	p.phoneNumber = row.PhoneNumber;

// Create relationship Customer OWNS Phone
LOAD CSV WITH HEADERS
FROM 'file://owns_phone.csv'  AS row
MATCH (c:Customer {name: row.Customer})
MATCH (p:Phone {phoneNumber: row.PhoneNumber})
MERGE (c)-[r:OWNS_PHONE]->(p);


// CREATE Email
CREATE CONSTRAINT Email_email IF NOT EXISTS 
FOR (x:Email) REQUIRE x.email IS UNIQUE;
LOAD CSV WITH HEADERS FROM 'file:///emails.csv' AS row
MERGE (p:Email {email: row.Email})
SET 
	p.email = row.Email;

// Create relationship Customer OWNS Email
LOAD CSV WITH HEADERS
FROM 'file:///owns_email.csv'  AS row
MATCH (c:Customer {name: row.Customer})
MATCH (p:Email {email: row.Email})
MERGE (c)-[r:OWNS_EMAIL]->(p);
```

### 2. Claim, Relationships with Customer, Claim embedding

```Cypher
// CREATE Claim
CREATE CONSTRAINT Claim_claimId IF NOT EXISTS 
FOR (x:Claim) REQUIRE x.claimId IS UNIQUE;

LOAD CSV WITH HEADERS FROM 'file:///claims.csv' AS row
MERGE (c:Claim {claimId: row.ClaimID})
SET
  c.claimId = row.ClaimID,
  c.risk = row.Risk,
  c.admissionDate = date(datetime({epochMillis: apoc.date.parse(row.AdmissionDate, 'ms', 'd-MMM-yy')})),
  c.age=row.Age,
  c.disease = row.Disease,
  c.los = toInteger(row.Los),
  c.fraud = row.Fraud,
  c.narration = row.Narration;
  
  
// Create relationship Customer FILED Claim
LOAD CSV WITH HEADERS
FROM 'file:///filed_claim.csv'  AS row
MATCH (c:Customer {name: row.Customer})
MATCH (clm:Claim {claimId: row.ClaimID})
MERGE (c)-[r:FILED_CLAIM]->(clm);

// Create Claim Embedding
LOAD CSV WITH HEADERS
FROM 'file:///openai_embedding_3small.csv'
AS row
MATCH (c:Claim {claimId: row.ClaimID})
CALL db.create.setNodeVectorProperty(c, 'embedding', apoc.convert.fromJsonList(row.NarrationEmbedding))
RETURN count(*);

// 1536 for ada002 and 3-small, 3072 for 3-large
CALL db.index.vector.createNodeIndex(
    'claimNarration',
    'Claim',
    'embedding',
    1536,
    'cosine'
);

```

**Important Note**

`claimNarration` - The name of the index

`Claim` - The node label on which to index

`embedding` - The property key on which to index

`1536` - The dimension of the embedding e.g. OpenAI embeddings consist of 1536 dimensions.

`cosine` - The similarity function to use when comparing values in this index - this can be euclidean or cosine.



### 3. Hospital, Relationship with Claim

```Cypher
// CREATE Hospital
CREATE CONSTRAINT Hospital_name IF NOT EXISTS 
FOR (x:Hospital) REQUIRE x.name IS UNIQUE;
LOAD CSV WITH HEADERS FROM 'file:///hospitals.csv' AS row
MERGE (c:Hospital {name: row.Hospital})
SET 
	c.name = row.Hospital;
	

// Create relationship Hospital PROVIDED_MEDICAL_SERVICE Claim
LOAD CSV WITH HEADERS
FROM 'file:///provided_medical.csv'  AS row
MATCH (h:Hospital {name: row.Hospital})
MATCH (clm:Claim {claimId: row.ClaimID})
MERGE (h)-[r:PROVIDED_MEDICAL_SERVICE]->(clm);
```

### 4. Agent, Relationship with 

```Cypher
// CREATE Agent
CREATE CONSTRAINT Agent_name IF NOT EXISTS 
FOR (x:Agent) REQUIRE x.name IS UNIQUE;
LOAD CSV WITH HEADERS FROM 'file:///agents.csv' AS row
MERGE (a:Agent {name: row.Agent})
SET 
	a.name = row.Agent;
	
// Create relationship Agent SERVICED Claim
LOAD CSV WITH HEADERS
FROM 'file:///serviced_claim.csv'  AS row
MATCH (a:Agent {name: row.Agent})
MATCH (clm:Claim {claimId: row.ClaimID})
MERGE (a)-[r:SERVICED_CLAIM]->(clm);
```

## Verify the data has been successfully imported by querying the database.

```Cypher
call db.schema.visualization()
```

<img src="../img/browser-schema.png" style="width:60%;height:60%;">



To access the created neo4j graph database via `Python` application, you would required to config the following environment variables:

```bash
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="<password>"
```

# Neo4j AuraDB - FreeTier

Alternative to `Neo4j Desktop`, you may choose to host a database on `Neo4j AuraDB` fully-managed-cloud services. Please follow the [Getting Started with AuraDB](https://neo4j.com/cloud/platform/aura-graph-database/?ref=docs-nav-get-started) to create an account and a free instance. Once, you have completed the registration and created a free `instance` - you may have only 1 instance for a free tier. You may use the same script to create `Node` and `Relationship`, or use [Data Importer Tool](https://www.youtube.com/watch?v=JXU7TxC_ExI)

Note that to use the provided script to create `Node` and `Relationhips` you need to host csv files via platforms like GitHub or Google Shared Drive, and change the url in the script as according to remot-hosted files. The following code shows how to run Cypher script via `Neo4j-Aura` on `Query` Panel.

<img src="../img/neo4j-auradb-import.png" style="width:60%;height:60%;">



**Full scripts**

```Cypher
// CREATE Customer
CREATE CONSTRAINT Customer_name IF NOT EXISTS 
FOR (x:Customer) REQUIRE x.name IS UNIQUE;
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/swatakit/graph-data/main/customers.csv' AS row
MERGE (c:Customer {name: row.Customer})
SET 
	c.name = row.Customer,
	c.gender = row.Gender;
	
	
// CREATE Phone
CREATE CONSTRAINT Phone_phoneNumber IF NOT EXISTS 
FOR (x:Phone) REQUIRE x.phoneNumber IS UNIQUE;
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/swatakit/graph-data/main/phones.csv' AS row
MERGE (p:Phone {phoneNumber: row.PhoneNumber})
SET 
	p.phoneNumber = row.PhoneNumber;

// Create relationship Customer OWNS Phone
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/swatakit/graph-data/main/owns_phone.csv'  AS row
MATCH (c:Customer {name: row.Customer})
MATCH (p:Phone {phoneNumber: row.PhoneNumber})
MERGE (c)-[r:OWNS_PHONE]->(p);


// CREATE Email
CREATE CONSTRAINT Email_email IF NOT EXISTS 
FOR (x:Email) REQUIRE x.email IS UNIQUE;
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/swatakit/graph-data/main/emails.csv' AS row
MERGE (p:Email {email: row.Email})
SET 
	p.email = row.Email;

// Create relationship Customer OWNS Email
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/swatakit/graph-data/main/owns_email.csv'  AS row
MATCH (c:Customer {name: row.Customer})
MATCH (p:Email {email: row.Email})
MERGE (c)-[r:OWNS_EMAIL]->(p);

// CREATE Claim
CREATE CONSTRAINT Claim_claimId IF NOT EXISTS 
FOR (x:Claim) REQUIRE x.claimId IS UNIQUE;

LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/swatakit/graph-data/main/claims.csv' AS row
MERGE (c:Claim {claimId: row.ClaimID})
SET
  c.claimId = row.ClaimID,
  c.risk = row.Risk,
  c.admissionDate = date(datetime({epochMillis: apoc.date.parse(row.AdmissionDate, 'ms', 'd-MMM-yy')})),
  c.age=row.Age,
  c.disease = row.Disease,
  c.los = toInteger(row.Los),
  c.fraud = row.Fraud,
  c.narration = row.Narration;
  
  
// Create relationship Customer FILED Claim
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/swatakit/graph-data/main/filed_claim.csv'  AS row
MATCH (c:Customer {name: row.Customer})
MATCH (clm:Claim {claimId: row.ClaimID})
MERGE (c)-[r:FILED_CLAIM]->(clm);

// Create Claim Embedding
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/swatakit/graph-data/main/openai_embedding_3small.csv'
AS row
MATCH (c:Claim {claimId: row.ClaimID})
CALL db.create.setNodeVectorProperty(c, 'embedding', apoc.convert.fromJsonList(row.NarrationEmbedding))
RETURN count(*);

// 1536 for ada002 and 3-small, 3072 for 3-large
CALL db.index.vector.createNodeIndex(
    'claimNarration',
    'Claim',
    'embedding',
    1536,
    'cosine'
);


// CREATE Hospital
CREATE CONSTRAINT Hospital_name IF NOT EXISTS 
FOR (x:Hospital) REQUIRE x.name IS UNIQUE;
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/swatakit/graph-data/main/hospitals.csv' AS row
MERGE (c:Hospital {name: row.Hospital})
SET 
	c.name = row.Hospital;
	

// Create relationship Hospital PROVIDED_MEDICAL_SERVICE Claim
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/swatakit/graph-data/main/provided_medical.csv'  AS row
MATCH (h:Hospital {name: row.Hospital})
MATCH (clm:Claim {claimId: row.ClaimID})
MERGE (h)-[r:PROVIDED_MEDICAL_SERVICE]->(clm);


// CREATE Agent
CREATE CONSTRAINT Agent_name IF NOT EXISTS 
FOR (x:Agent) REQUIRE x.name IS UNIQUE;
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/swatakit/graph-data/main/agents.csv' AS row
MERGE (a:Agent {name: row.Agent})
SET 
	a.name = row.Agent;
	
// Create relationship Agent SERVICED Claim
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/swatakit/graph-data/main/serviced_claim.csv'  AS row
MATCH (a:Agent {name: row.Agent})
MATCH (clm:Claim {claimId: row.ClaimID})
MERGE (a)-[r:SERVICED_CLAIM]->(clm);
```